#Knights and Knaves puzzles <br/>
This project Used to enhance understanding of the knowledge and Reasoning concepts to solve logic puzzles<br/>
The idea is based on Raymond Smullyan's book, "What is the name of this book?" which contains “Knights and Knaves” puzzles.<br/>

In a Knights and Knaves puzzle, the following information is given:<br/>
- Each character is either a knight or a knave.
- A knight will always tell the truth: if a knight states a sentence, then that sentence is true.
- Conversely, a knave will always lie: if a knave states a sentence, then that sentence is false.

The objective of the puzzle is, given a set of sentences spoken by each of the characters, determine, for each character, whether that character is a knight or a knave.<br/>
For example, consider a simple puzzle with just a single character named A. A says “I am both a knight and a knave.”<br/>
Logically, we might reason that if A were a knight, then that sentence would have to be true. But we know that the sentence cannot possibly be true, because A cannot be both a knight and a knave; we know that each character is either a knight or a knave, but not both. So, we could conclude, A must be a knave.

#Logic
the logic cell below was provided by my computer science unit Lecturer and helps to define several classes for different types of logical connectives such as `And(A,B)`, `Or(A,B)`, `Not(Or(A,B))` which will be used in the knowledge base to connect various 'knowledges' in order to provide appropriate reasoning

In [1]:
#@title Logic
# -*- coding: utf-8 -*-
import itertools


class Sentence():

    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0
        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            # return f"({s})"
            return "({})".format(s)


class Symbol(Sentence):

    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise Exception("variable {} not in model".format(self.name))

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}


class Not(Sentence):
    def __init__(self, operand):
        Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return "Not({})".format(self.operand)

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()


class And(Sentence):
    def __init__(self, *conjuncts):
        for conjunct in conjuncts:
            Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(
            ("and", tuple(hash(conjunct) for conjunct in self.conjuncts))
        )

    def __repr__(self):
        conjunctions = ", ".join(
            [str(conjunct) for conjunct in self.conjuncts]
        )
        return "And({})".format(conjunctions)

    def add(self, conjunct):
        Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " ∧ ".join([Sentence.parenthesize(conjunct.formula())
                           for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])


class Or(Sentence):
    def __init__(self, *disjuncts):
        for disjunct in disjuncts:
            Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(
            ("or", tuple(hash(disjunct) for disjunct in self.disjuncts))
        )

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return "Or({})".format(disjuncts)

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " ∨  ".join([Sentence.parenthesize(disjunct.formula())
                            for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])


class Implication(Sentence):
    def __init__(self, antecedent, consequent):
        Sentence.validate(antecedent)
        Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication)
                and self.antecedent == other.antecedent
                and self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return "Implication({}, {})".format(self.antecedent, self.consequent)

    def evaluate(self, model):
        return ((not self.antecedent.evaluate(model))
                or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return "{} => {}".format(antecedent, consequent)

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())


class Biconditional(Sentence):
    def __init__(self, left, right):
        Sentence.validate(left)
        Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional)
                and self.left == other.left
                and self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return "Biconditional({}, {})".format(self.left, self.right)

    def evaluate(self, model):
        return ((self.left.evaluate(model)
                 and self.right.evaluate(model))
                or (not self.left.evaluate(model)
                    and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(str(self.left))
        right = Sentence.parenthesize(str(self.right))
        return "{} <=> {}".format(left, right)

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())


def model_check(knowledge, query):
    """Checks if knowledge base entails query."""

    def check_all(knowledge, query, symbols, model):
        """Checks if knowledge base entails query, given a particular model."""

        # If model has an assignment for each symbol
        if not symbols:

            # If knowledge base is true in model, then query must also be true
            if knowledge.evaluate(model):
                return query.evaluate(model)
            return True
        else:

            # Choose one of the remaining unused symbols
            remaining = symbols.copy()
            p = remaining.pop()

            # Create a model where the symbol is true
            model_true = model.copy()
            model_true[p] = True

            # Create a model where the symbol is false
            model_false = model.copy()
            model_false[p] = False

            # Ensure entailment holds in both models
            return (check_all(knowledge, query, remaining, model_true) and
                    check_all(knowledge, query, remaining, model_false))

    # Get all symbols in both knowledge and query
    symbols = set.union(knowledge.symbols(), query.symbols())

    # Check that knowledge entails query
    return check_all(knowledge, query, symbols, dict())

#symbols
In the code cell below, six propositional symbols are defined. AKnight, for example, represents the sentence that “A is a knight,” </br>


In [2]:
AKnight = Symbol("A is a Knight")
AKnave = Symbol("A is a Knave")

BKnight = Symbol("B is a Knight")
BKnave = Symbol("B is a Knave")

CKnight = Symbol("C is a Knight")
CKnave = Symbol("C is a Knave")  # ie, "C is a knave"

#Puzzle solver
In the `puzzle_solver()` function below, the symbols are specified in a list. The model checking function is then used to compute, given the knowledge for that puzzle, whether each character is a knight or a knave, printing out any conclusions that the model checking algorithm is able to make. This code was also provided from class

In [3]:
def puzzle_solver(puzzle_name, knowledge):
    symbols = [AKnight, AKnave, BKnight, BKnave, CKnight, CKnave]
    print(puzzle_name)

    if len(knowledge.conjuncts) == 0:
        print("    Not yet implemented.")
    else:
        for symbol in symbols:
            if model_check(knowledge, symbol):
                print(f"    {symbol}")

# The actual Puzzles and Knowledge bases
In the cells below, there are six different knowledge bases, `knowledge0`, `knowledge1`, `knowledge2`, `knowledge3`, `knowledge4`,and `knowledge5`, which will contain the knowledge needed to deduce the solutions to the Puzzles 0, 1, 2, 3, 4 and 5, respectively</br>
To implement this is how I reasoned
- I first assumed that A, B and C are people speaking
- I then included the cases where each 'person' is knave and when is knight
- Then used the recollection of their speeches  to create an implication, to see who is lying, and whos saying the truth
- The implication compares their speeches and their personal sincerety

##Puzzle 0
- A says “I am both a knight and a knave.”

In [ ]:
knowledge0 = And(
    And(Or(AKnave,AKnight),
    Not(And(AKnave,AKnight))),

    Implication(AKnave,Not(And(AKnight,AKnave))),
    Implication(AKnight,(And(AKnight,AKnave)))
)

# Solve the puzzle
puzzle_solver('Puzzle 0', knowledge0)

## Puzzle 1
**Puzzle 1** has two characters: A and B.
- A says “We are both knaves.”
- B says nothing.

In [ ]:
knowledge1 = And(
    # YOUR CODE HERE
    And(Or(AKnave,AKnight), Not(And(AKnave,AKnight))),
    And(Or(BKnave,BKnight), Not(And(BKnave,BKnight))),
    Implication(AKnave,Not(And(AKnave,BKnave))),
    Implication(AKnight,(And(AKnave,BKnave)))
)

# Solve the puzzle
puzzle_solver('Puzzle 1', knowledge1)

## Puzzle 2
**Puzzle 2** has two characters: A and B.
- A says “We are the same kind.”
- B says “We are of different kinds.”

In [ ]:
knowledge2 = And(
    # YOUR CODE HERE
    And(Or(AKnave,AKnight), Not(And(AKnave,AKnight))),
    And(Or(BKnave,BKnight), Not(And(BKnave,BKnight))),
    Implication(AKnight,Or(And(AKnave,BKnave),And(AKnight,BKnight))),
    Implication(AKnave,Not(Or(And(AKnave,BKnave),And(AKnight,BKnight)))),
    Implication(BKnight,Or(And(AKnave,BKnight),And(BKnave,AKnight))),
    Implication(BKnave,Not(Or(And(AKnave,BKnight),And(BKnave,AKnight))))
)

# Solve the puzzle
puzzle_solver('Puzzle 2', knowledge2)

## Puzzle 3
**Puzzle 3** has three characters: A, B, and C.
- A says either “I am a knight.” or “I am a knave.”, but you don’t know which.
- B says “A said ‘I am a knave.’”
- B then says “C is a knave.”
- C says “A is a knight.”

In [ ]:
statement_a = Or(Implication(AKnave, AKnave), Implication(AKnight, AKnave))
statement_b = And(Implication(AKnave, AKnave),Implication(AKnight, AKnave))
statement_c = And(statement_a, Not(statement_b))
statement_d = Or(Implication(AKnave, AKnight), Implication(AKnight, AKnight))

knowledge3 = And(
    And(Or(AKnave,AKnight), Not(And(AKnave,AKnight))),
    And(Or(BKnave,BKnight), Not(And(BKnave,BKnight))),
    And(Or(CKnave,CKnight), Not(And(CKnave,CKnight))),

    Implication(AKnight, And((Or(AKnave,AKnight)),Not(And(AKnave,AKnight)))),
    Implication(AKnave, And((Or(AKnave,AKnight)),Not(And(AKnave,AKnight)))),
    Implication(CKnight, AKnight),
    Implication(CKnave, AKnave),
    Implication(BKnight, And(CKnave,statement_c)),
    Implication(BKnave, And(Not(CKnave),(statement_d)))

)

puzzle_solver('Puzzle 3', knowledge3)

## Puzzle 4
**Puzzle 4** has three characters: A, B, and C.
- A says “B never lies.”
- C says “A is a knave and I am a knave.”
- B says “C is a knave.”

In [ ]:
knowledge4 = And(
    # YOUR CODE HERE
    And(Or(AKnave,AKnight), Not(And(AKnave,AKnight))),
    And(Or(BKnave,BKnight), Not(And(BKnave,BKnight))),
    And(Or(CKnave,CKnight), Not(And(CKnave,CKnight))),

    Implication(AKnight,BKnight),
    Implication(AKnave,Not(BKnight)),
    Implication(CKnight,And(AKnave,CKnave)),
    Implication(CKnave,Not(And(AKnave,CKnave))),
    Implication(BKnight,CKnave),
    Implication(BKnave,Not(CKnave))

)

# Solve the puzzle
puzzle_solver('Puzzle 4', knowledge4)

## Puzzle 5
**Puzzle 5** has three characters: A, B, and C.
- B says “A is like me”
- A says “C always tells the truth.”
- C says “B lies.”
- B says “A is lying.”

In [ ]:
knowledge5 = And(
    # YOUR CODE HERE
    And(Or(AKnave,AKnight), Not(And(AKnave,AKnight))),
    And(Or(BKnave,BKnight), Not(And(BKnave,BKnight))),
    And(Or(CKnave,CKnight), Not(And(CKnave,CKnight))),

    Implication(BKnight, And(AKnight,AKnave)),
    Implication(BKnave, Not(And(AKnight,AKnave))),

    Implication(AKnight, CKnight),
    Implication(AKnave, Not(CKnight)),
    Implication(CKnight, BKnave),
    Implication(CKnave, Not(BKnave))
)

# Solve the puzzle
puzzle_solver('Puzzle 5', knowledge5)